# Superación del problema de las estimaciones sesgadas al analizar datos de mercados abiertos mediante el método de remuestreo Jackknife

Cyrill A. Murashev, 2023-02-05

## Resumen

Los tasadores se enfrentan a menudo a la necesidad de analizar y describir datos de mercado recogidos en los mercados abiertos. Casi siempre no pueden obtener los datos de todo el mercado, sino que tratan con muestras, que pueden ser muy pequeñas en relación con toda la población general. En este caso, surge el problema de las estimaciones sesgadas. De lo anterior se deduce que cualquier estimación estadística realizada a partir de la muestra en cuestión es una estimación para la propia muestra. Al mismo tiempo, puede tener un sesgo en relación con la estimación que se obtendría en el caso de un análisis de toda la población general. Los tasadores suelen decir que han calculado algunas estadísticas descriptivas del mercado. Puede tratarse de la media o mediana del precio, el máximo y el mínimo, etc. Pero debemos entender que son sólo estimaciones para muestras, no para todo el mercado. Hoy estudiaremos las bases teóricas mínimas del método. Y luego lo aplicaremos a datos reales del mercado utilizando el lenguaje Python. Aprenderemos a determinar para cualquier estimación si existe sesgo y a reducir automáticamente su componente lineal. Este documento tiene versiones en [inglés](https://github.com/Kirill-Murashev/AI_for_valuers_book/blob/main/Parts-Chapters/Jackknife/jackknife.ipynb), [español](https://github.com/Kirill-Murashev/AI_for_valuers_book/blob/main/Parts-Chapters/Jackknife/jackknife-esp.ipynb) y [ruso](https://github.com/Kirill-Murashev/AI_for_valuers_book/blob/main/Parts-Chapters/Jackknife/jackknife-nov.ipynb). La versión más actualizada y rápida es la inglesa. Si hay discrepancias entre versiones, debe confiarse en la versión inglesa.

## Bases del método Jackknife

### Introducción

En primer lugar, debemos recordar por qué los tasadores necesitan estadísticas. Suelen tener una cierta distribución de las características de los objetos a partir de la muestra de análogos recogidos en el mercado abierto. Y tratan de obtener algunas estimaciones de los valores de esas características. Puede ser la media, la mediana, el máximo, el mínimo, la varianza, etc. A veces también necesitan comparar dos o más submuestras para decidir si es necesario realizar algunos ajustes en función de la diferencia de valores de las características. Como podemos adivinar, la mayoría de las veces los tasadores trabajan con muestras, no con todo el mercado. Por lo tanto, los tasadores sólo pueden obtener estimaciones muestrales de los valores de las características, pero no sus valores reales. 
El método de Jackknife puede gestionar dos cuestiones:
- reducir el sesgo de la estimación muestral con respecto al valor real de la población general;
- calcular la varianza del valor ajustado de la característica.

Supongamos que tenemos alguna característica *X* (podría ser el precio unitario, por ejemplo), cuya distribución en la población general desconocemos. Pero tenemos una muestra formada por n elementos $[x{1},\ldots, x_{n}]$. Queremos estimar la expectativa de *X* que se puede escribir como $\mathbb{E}[X]$. En general, la expectativa puede expresarse del siguiente modo
$$\mathbb{E}[X] = \sum_{j=1}^{n >> 1} p(x_{j})x_{j}.$$

Pero sólo tenemos una muestra que, por supuesto, consiste en un número muy limitado de observaciones, lejos del infinito. Por tanto, no podemos estimar la expectativa, sólo la media muestral que se anota como
$$\hat{\mu}=\dfrac{1}{n} \sum_{i=1}^{n<<\infty}x_{i}.$$

Por lo tanto, no utilizamos probabilidades, sino frecuencias observadas. Es evidente que $\mathbb{E}[X] \neq \hat{\mu}$, pero $\hat{\mu} = \mathbb{E}[X] + \mathcal{bias}$, donde $\mu$ es la estimación de la expectativa, y el sesgo es algún desplazamiento sistemático entre los valores verdadero y estimado de la expectativa.


### Concepto general del método Jacknife

Hemos considerado un caso especial. Ahora podemos pasar al concepto más general de sesgo del estimador. Consideremos la variable aleatoria *X* que tiene la distribución desconocida *U*. Existe un parámetro de su distribución denominado $\theta$. Y nuestro objetivo es determinar su valor. El uso del parámetro abstracto $\theta$ en lugar de uno específico pone de relieve la universalidad del método Jackknife, que es capaz de detectar sesgos para cualquier parámetro de la distribución y corregir automáticamente su componente lineal. También tenemos el parámetro $\hat{\theta}$, que es la estimación de muestreo obtenida al utilizar alguna función. Debido al hecho de que $\hat{\theta}$ se obtuvo de una muestra, mientras que nuestro objetivo es estimar $\theta$ para la población general, es decir, todo el mercado en el contexto de la valoración, $\hat{\theta}$ tiene un sesgo en relación con $\theta$. Matemáticamente, significa que la expectativa para $\hat{\theta}$ no es igual a la expectativa para $\theta$:
$$\mathbb{E}(\hat{\theta}) \neq \mathbb{E}(\theta).$$
En este caso, podemos decir que
$$\mathbb{E}(\hat{\theta}_{n}) = \theta + \frac{\alpha}{n} + \frac{\beta}{n^{2}} + \frac{\gamma}{n^{3}} + \ldots \frac{\omega}{n^{(k\rightarrow \infty)}},$$
donde $\theta$ es el valor verdadero del parámetro para la población general, e $\frac{\alpha}{n} + \frac{\beta}{n^{2}} + \frac{\gamma}{n^{3}} + \ldots \frac{\omega}{n^{(k\rightarrow \infty)}}$ son los componentes lineales, cuadráticos, cúbicos y otros del sesgo. Todos los componentes disminuyen con el crecimiento de la muestra de acuerdo con funciones lineales, cuadráticas, cúbicas y otras. El término lineal introduce el mayor error porque es el que disminuye más lentamente de todos los demás.

El método Jackknife elimina el componente lineal del sesgo. Introduzcamos nuevas definiciones.

$\hat{\theta}_{i}$ es el valor de $\hat{\theta}$ que se obtendría al calcular no sobre una muestra completa, sino sobre una muestra con una observación excluida *i*, que toma valores de 1 a *n*. Entonces
$$\mathbb{E}(\hat{\theta}_{(i)}) = \theta + \frac{\alpha}{n-1} + \frac{\beta}{(n-1)^{2}} + \frac{\gamma}{(n-1)^{3}} + \ldots \frac{\omega}{(n-1)^{(k\rightarrow \infty)}}.$$
$\overline{\theta}$ es el valor medio de todos los $\hat{\theta}_{i}$.
$$\overline{\theta} = \frac{1}{n} \sum_{i=1}^{n} \hat{\theta}_{i}$$


### Sumario

Por lo tanto, para aplicar el método Jackknife, es decir, detectar la presencia de un sesgo y eliminar automáticamente su componente lineal, se requiere el siguiente conjunto de pasos.
1. Necesitamos estimar algún parámetro $\theta$ de una variable aleatoria *X*.
1. Obtengamos alguna estimación de $\hat{\theta}$ para la muestra utilizando alguna función matemática $\hat{\theta}=F(x_{1},\ldots,x_{n})$.
1. $\hat{\theta}$ puede ser parcial.
1. $\theta = \mathbb{E}(\hat{\theta}) + bias$.
1. Creemos las nuevas *n* muestras mediante la exclusión secuencial de una *x* de la muestra inicial.
1. Calcular la $\hat{\theta}_{(i)}$ para todas las nuevas muestras utilizando la misma función **F**.
1. Calcula la media de todas las $\hat{\theta}_{(i)}$ y dénotala como $\overline{\theta}$.
1. Calcule el sesgo mediante la siguiente fórmula
$$\widehat{bias}_{jack} = (n-1)(\overline{\theta} - \hat{\theta}).$$
1. Elimine el componente lineal del sesgo mediante la fórmula
$$\hat{\theta}_{jacked} = \hat{\theta} - \widehat{bias}_{jack}.$$

## Aplicación práctica en Python

Hoy utilizaremos un conjunto de datos que contiene 34821 observaciones del mercado inmobiliario residencial de San Petersburgo. Se obtuvo del web-scrapping en septiembre de 2021. Supongamos que este conjunto de datos contiene datos sobre todo el mercado, por lo que podemos utilizarlo como población general. A continuación, crearemos una submuestra que contenga sólo 25 observaciones, lo que refleja el número típico de observaciones que maneja un tasador. Calcularemos “la expectativa” de nuestra “población general” y, a continuación, calcularemos la media de la muestra. Por último, aplicaremos el método Jackknife y compararemos su resultado del cálculo de la media en relación con la media muestral.

In [13]:
# import libraries
import numpy as np
import pandas as pd
from astropy.stats import jackknife_resampling
from astropy.stats import jackknife_stats
from random import sample

In [52]:
# import data
df = pd.read_csv("spba-flats-210928.csv", index_col=False)
print(df)

       Unnamed: 0                                     links   price_m  county
0               1  https://spb.cian.ru/sale/flat/262765174/  155460.0  sadadm
1               2  https://spb.cian.ru/sale/flat/263280601/  295455.0  sadadm
2               3  https://spb.cian.ru/sale/flat/261612519/  310559.0  sadadm
3               4  https://spb.cian.ru/sale/flat/263094016/  100000.0  sadadm
4               5  https://spb.cian.ru/sale/flat/262339898/  145929.0  sadadm
...           ...                                       ...       ...     ...
34816       34817  https://spb.cian.ru/sale/flat/256621764/   70093.0  llobol
34817       34818  https://spb.cian.ru/sale/flat/261430727/   67227.0  llobol
34818       34819  https://spb.cian.ru/sale/flat/246538655/   86207.0  llobol
34819       34820  https://spb.cian.ru/sale/flat/246587468/   65455.0  llobol
34820       34821  https://spb.cian.ru/sale/flat/239698989/   89041.0  llobol

[34821 rows x 4 columns]


In [53]:
# calculate mean and maximum for the "general population"
gp_mean = df['price_m'].mean()
gp_max = df['price_m'].max()
print("The mean of unit price for 'general population' is", gp_mean)
print("The maximum of unit price for 'general population' is", gp_max)

The mean of unit price for 'general population' is 176116.52505671864
The maximum of unit price for 'general population' is 1624829.0


In [68]:
# create sample
sam_size = 25
ran_sam = df.sample(n=sam_size)
print(ran_sam)

       Unnamed: 0                                     links   price_m  county
20606       20607  https://spb.cian.ru/sale/flat/264064476/  213889.0  sprkol
11160       11161  https://spb.cian.ru/sale/flat/261519367/  161667.0  skryuz
21112       21113  https://spb.cian.ru/sale/flat/263895261/  148830.0  sprlax
19612       19613  https://spb.cian.ru/sale/flat/250041085/  189706.0  sprn65
5076         5077  https://spb.cian.ru/sale/flat/263660571/  139382.0  skapis
13890       13891  https://spb.cian.ru/sale/flat/263013521/  231935.0  smonow
12315       12316  https://spb.cian.ru/sale/flat/262914352/  198765.0  smozwy
8250         8251  https://spb.cian.ru/sale/flat/263518941/  181985.0  skrpol
243           244  https://spb.cian.ru/sale/flat/251281327/  210697.0  sadeka
15608       15609  https://spb.cian.ru/sale/flat/260189091/  134454.0  snenew
7387         7388  https://spb.cian.ru/sale/flat/257597416/  108974.0  skomet
5179         5180  https://spb.cian.ru/sale/flat/261053782/  150

In [67]:
# calculate mean and maximum for the samle
rs_mean = ran_sam['price_m'].mean()
rs_max = ran_sam['price_m'].max()
print("The mean of unit price for random sample is", rs_mean)
print("The maximum of unit price for random sample is", rs_max)

The mean of unit price for random sample is 147030.5
The maximum of unit price for random sample is 181481.0


In [69]:
# obtain Jackknife resamples
new_df = ran_sam["price_m"]
array = new_df.to_numpy()
print(new_df)

20606    213889.0
11160    161667.0
21112    148830.0
19612    189706.0
5076     139382.0
13890    231935.0
12315    198765.0
8250     181985.0
243      210697.0
15608    134454.0
7387     108974.0
5179     150000.0
22767    121429.0
22444    223214.0
10338    177384.0
13155    195000.0
804      138900.0
18970    221212.0
21243    233333.0
28099    110754.0
31551     83459.0
4399     151282.0
12014    198780.0
22696    145516.0
23239    204433.0
Name: price_m, dtype: float64


In [70]:
# obtain Jackknife resamples
resamples = jackknife_resampling(array)
print(resamples)

[[161667. 148830. 189706. 139382. 231935. 198765. 181985. 210697. 134454.
  108974. 150000. 121429. 223214. 177384. 195000. 138900. 221212. 233333.
  110754.  83459. 151282. 198780. 145516. 204433.]
 [213889. 148830. 189706. 139382. 231935. 198765. 181985. 210697. 134454.
  108974. 150000. 121429. 223214. 177384. 195000. 138900. 221212. 233333.
  110754.  83459. 151282. 198780. 145516. 204433.]
 [213889. 161667. 189706. 139382. 231935. 198765. 181985. 210697. 134454.
  108974. 150000. 121429. 223214. 177384. 195000. 138900. 221212. 233333.
  110754.  83459. 151282. 198780. 145516. 204433.]
 [213889. 161667. 148830. 139382. 231935. 198765. 181985. 210697. 134454.
  108974. 150000. 121429. 223214. 177384. 195000. 138900. 221212. 233333.
  110754.  83459. 151282. 198780. 145516. 204433.]
 [213889. 161667. 148830. 189706. 231935. 198765. 181985. 210697. 134454.
  108974. 150000. 121429. 223214. 177384. 195000. 138900. 221212. 233333.
  110754.  83459. 151282. 198780. 145516. 204433.]
 [213

In [71]:
# obtain Jackknife resamples shape
resamples.shape

(25, 24)

In [77]:
# obtain Jackknife estimate for the mean, its bias,
# its standard error, and its 95% confidence interval
test_statistic = np.mean

estimate, bias, stderr, conf_interval = jackknife_stats(
    array, test_statistic, 0.95)

mean_jacked = estimate
print("the jacked mean is ", mean_jacked)
bias_jack = abs(mean_jacked - rs_mean)
print("the bias got from the Jackknife is ", bias_jack)
std_error = stderr
print("the standard error got from the Jackknife is ", std_error)
conf_int = conf_interval
print("the confident interval (95%) of jacked mean is ", conf_int)

the jacked mean is  170999.2
the bias got from the Jackknife is  23968.70000000001
the standard error got from the Jackknife is  8468.584368318783
the confident interval (95%) of jacked mean is  [154401.07963806 187597.32036194]


Aomo podemos ver, la expectativa es 176116,525, la media muestral es 147030,500 y la media ajustada obtenida por el método Jackknife es 170999,200. El intervalo de confianza para la expectativa es [154401.080, 187597.320] con una probabilidad de 0,95. Así, demostramos de forma inequívoca que la aplicación del método Jackknife conduce a una mejora significativa de la precisión de la estimación del parámetro de distribución. Además, el intervalo de confianza calculado incluye el verdadero valor de la expectativa.


## Epílogo

El método Jackknife es una herramienta sencilla y computacionalmente eficaz para el ajuste de los estimadores muestrales. Espero que su aplicación ayude a muchos tasadores en su práctica diaria. Y tal vez inspire a alguien a utilizar más ampliamente los métodos de aprendizaje automático en las actividades de valoración.